Data set available at [this link](https://economistsview.typepad.com/economics421/files/Data-for-Problem-3.xls)

Implementation inspired by the solution presented at [this link](https://economistsview.typepad.com/economics421/2012/02/solution-to-homework-3.html)

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('../Lecture04/Data-for-Problem-3.xls', usecols=[1,2])

In [3]:
data.head()

,SALARY,YEARS
0,105.2,36
1,91.3,30
2,72.5,29
3,74.3,28
4,103.5,26


Linear model:

$$
\ln \textrm{ Salary} = \beta_1 + \beta_2 \textrm{ years} + \beta_3 \textrm{ years}^2 + u_i
$$

In [4]:
data['LOG_SALARY'] = data['SALARY'].apply(np.log)

In [5]:
data['YEARS_SQ'] = data['YEARS'].pow(2)

In [6]:
data.head()

,SALARY,YEARS,LOG_SALARY,YEARS_SQ
0,105.2,36,4.655863,1296
1,91.3,30,4.514151,900
2,72.5,29,4.283587,841
3,74.3,28,4.308111,784
4,103.5,26,4.639572,676


___

1. Regress $y$ on constant, $\textrm{ years}, \textrm{ years}^2$

In [7]:
from sklearn.linear_model import LinearRegression

In [8]:
linreg = LinearRegression()

In [9]:
X = data[['YEARS','YEARS_SQ']]
y = data['LOG_SALARY']

In [10]:
linreg.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [11]:
print(linreg.intercept_, linreg.coef_)

3.8093653545744024 [ 0.04385284 -0.00062735]


In [12]:
linreg.score(X,y)

0.5361786383088458

2. Calculate the estimated errors: $\hat u_i = y_i - \hat\beta_1 + \hat\beta_2 \textrm{ years } + \hat\beta_3 \textrm{ years}^2$ :

In [13]:
y_hat = linreg.predict(X)

In [14]:
u_hat = y - y_hat

Variance model:

$$
(c) \quad \ln\sigma_i^2 = \alpha_1 + \alpha_2 z_2 + \alpha_3 z_3 + ... + \alpha_p z_p
$$

3. Regress $\ln \hat u_i^2$ on constant, $z_1, ..., z_p$

In [15]:
ln_u_hat_sq = np.log(u_hat.pow(2))

In [16]:
varreg = LinearRegression()

In [17]:
Xvar = X.copy()
yvar = ln_u_hat_sq.copy()

In [18]:
varreg.fit(Xvar, yvar)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [19]:
print(varreg.intercept_, varreg.coef_)

-6.562664425653965 [ 0.23556209 -0.00477648]


4. Use the predicted variance $\hat{\ln \sigma^2}$ to obtain $\hat\sigma_i$:

In [20]:
ln_sigma_hat_sq = varreg.predict(Xvar)

In [21]:
sigma_hat_sq = np.exp(ln_sigma_hat_sq)

In [22]:
sigma_hat = np.sqrt(sigma_hat_sq)

5. Divide the whole original model by $\hat\sigma_i$:

In [23]:
data['LOG_SALARY_STAR'] = data['LOG_SALARY'].divide(sigma_hat)
data['C_STAR'] = np.reciprocal(sigma_hat)
data['YEARS_STAR'] = data['YEARS'].divide(sigma_hat)
data['YEARS_SQ_STAR'] = data['YEARS_SQ'].divide(sigma_hat)

In [24]:
data.head()

,SALARY,YEARS,LOG_SALARY,YEARS_SQ,LOG_SALARY_STAR,C_STAR,YEARS_STAR,YEARS_SQ_STAR
0,105.2,36,4.655863,1296,39.429072,8.468692,304.872908,10975.424695
1,91.3,30,4.514151,900,30.100314,6.667990,200.039714,6001.191415
2,72.5,29,4.283587,841,27.909969,6.515561,188.951267,5479.586755
3,74.3,28,4.308111,784,27.559411,6.397099,179.118765,5015.325406
4,103.5,26,4.639572,676,29.023289,6.255597,162.645513,4228.783348


In [25]:
fgls = LinearRegression(fit_intercept=False)

In [26]:
Xstar = data[['C_STAR','YEARS_STAR','YEARS_SQ_STAR']]
ystar = data['LOG_SALARY_STAR']

In [27]:
fgls.fit(Xstar, ystar)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)

In [28]:
print(fgls.coef_)

[ 3.82750788e+00  3.82145695e-02 -4.43338826e-04]


In [29]:
fgls.score(Xstar,ystar)

0.9898301975446464